In [1]:
import numpy as np
from mydx import mydx, fmm_pcm
#mydx.init()
#nbasis, ngrid, nsph = mydx.get_sizes()
#lmax = int(nbasis**0.5)-1
#print(nbasis, lmax, ngrid)

In [2]:
# COMPLEX
lmax = 2
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1
sph2_c = np.array([.0, 0, 0.5])
sph2_r = 1
src1 = np.array([0.0, 0.5, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2m_complex(src1-sph1_c, lmax)
coef2_m = fmm_pcm.p2m_complex(src1-sph2_c, lmax)
print(fmm_pcm.m2p_complex(dst1-sph1_c, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_complex(dst1-sph2_c, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)
coef1_m_normal = fmm_pcm.p2m_complex_normal(src1-sph1_c, lmax)
coef2_m_normal = fmm_pcm.p2m_complex_normal(src1-sph2_c, lmax)
print(fmm_pcm.m2p_complex_normal(dst1-sph1_c, lmax, coef1_m_normal), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_complex_normal(dst1-sph2_c, lmax, coef2_m_normal), np.linalg.norm(src1-dst1)**-1)
coef2_m_m2m = fmm_pcm.m2m_complex_translate_z(sph1_c[2]-sph2_c[2], lmax, coef1_m)
print(coef1_m)
print(coef2_m)
print(coef2_m_m2m)
print(fmm_pcm.m2p_complex_normal(dst1-sph2_c, lmax, coef2_m_m2m), np.linalg.norm(src1-dst1)**-1)

0.3127313958110473 0.31234752377721214
0.3126262395925993 0.31234752377721214
0.3127313958110473 0.31234752377721214
0.3126262395925993 0.31234752377721214
[ 1.        +0.j          0.        +0.35355339j  0.        +0.j
  0.        -0.35355339j -0.15309311+0.j          0.        +0.j
 -0.125     +0.j          0.        -0.j         -0.15309311-0.j        ]
[ 1.        +0.j          0.        +0.35355339j -0.5       +0.j
  0.        -0.35355339j -0.15309311+0.j         -0.        -0.30618622j
  0.125     +0.j         -0.        +0.30618622j -0.15309311-0.j        ]
[ 1.        +0.j          0.        +0.35355339j -0.5       +0.j
  0.        -0.35355339j -0.15309311+0.j          0.        -0.1767767j
  0.        +0.j          0.        +0.1767767j  -0.15309311+0.j        ]
1.116712476398384 0.31234752377721214


In [4]:
# REAL
lmax = 17
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1
sph2_c = np.array([1.0, 1, 1])
sph2_r = 1
src1 = np.array([0.0, 0.8, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2m_real(src1-sph1_c, lmax)
coef2_m = fmm_pcm.p2m_real(src1-sph2_c, lmax)
print(fmm_pcm.m2p_real(dst1-sph1_c, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_real(dst1-sph2_c, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)
coef1_m_normal = fmm_pcm.p2m_real_normal(src1-sph1_c, lmax)
coef2_m_normal = fmm_pcm.p2m_real_normal(src1-sph2_c, lmax)
print(fmm_pcm.m2p_real_normal(dst1-sph1_c, lmax, coef1_m_normal), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_real_normal(dst1-sph2_c, lmax, coef2_m_normal), np.linalg.norm(src1-dst1)**-1)
#coef2_m_m2m = fmm_pcm.m2m_complex_translate_z(sph1_c[2]-sph2_c[2], lmax, coef1_m)
#print(coef1_m)
#print(coef2_m)
#print(coef2_m_m2m)
#print(fmm_pcm.m2p_complex_normal(dst1-sph2_c, lmax, coef2_m_m2m), np.linalg.norm(src1-dst1)**-1)

0.3254722774521031 0.32547227745205964
0.32525817347924185 0.32547227745205964
0.3254722774521031 0.32547227745205964
0.3252581734792418 0.32547227745205964


In [45]:
# Compute M2M matrix by using P2M and M2P
lmax = 20
nparticles = 4000
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 1
sph2_c = np.array([1.0, 1, 1], dtype=np.float64)
sph2_r = 1
particles = np.zeros((nparticles, 3), dtype=np.float64)
mat1 = np.zeros((nparticles, (lmax+1)**2), dtype=np.float64)
mat2 = np.zeros((nparticles, (lmax+1)**2), dtype=np.float64)
for i in range(nparticles):
    tmp = np.random.randn(3)
    tmp = tmp / np.linalg.norm(tmp)
    particles[i] = sph1_c + 0.5 * sph1_r * tmp
    mat1[i] = fmm_pcm.p2m_real_normal(particles[i] - sph1_c, lmax)
    mat2[i] = fmm_pcm.p2m_real_normal(particles[i] - sph2_c, lmax)
q, r = np.linalg.qr(mat1)
#print(r)
m2m = np.linalg.solve(r, q.T @ mat2).T
print(np.linalg.norm(m2m @ mat1[0] - mat2[0]) / np.linalg.norm(mat2[0]))
for i in range(m2m.shape[0]):
    m2m[i, i+1:] = 0
#print(m2m)
print(np.linalg.norm(m2m @ mat1[0] - mat2[0]) / np.linalg.norm(mat2[0]))
print(m2m)

6.270042485324812e-13
5.729090017670565e-13
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.73205081e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.73205081e+00 -1.68616714e-16  1.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [-7.56897054e-11 -2.19908476e+05  6.44237818e-11 ...  1.00004674e+00
   0.00000000e+00  0.00000000e+00]
 [-1.03825705e+04 -1.13893217e+05  5.99437985e+03 ...  3.98367879e-05
   9.99963723e-01  0.00000000e+00]
 [-3.28325706e+03 -1.89558935e+04 -9.60148887e-12 ...  7.46396523e-06
  -4.26637865e-06  1.00000407e+00]]
